![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# TFM - Análisis Exploratorio de Datos (EDA)

### Descripción del dataset y su contexto:
El presente dataset contiene información detallada sobre propiedades en venta, incluyendo su precio, ubicación, dimensiones y diversas características adicionales. Cada registro representa un anuncio de un inmueble y cuenta con datos sobre su barrio, distrito, consumo energético y otros atributos socioeconómicos.

El objetivo de este estudio es analizar el conjunto de datos para extraer insights que permitan comprender mejor el mercado inmobiliario y detectar patrones que puedan ser útiles para la toma de decisiones.

### Objetivos del EDA:
1. Comprender la estructura del dataset: Identificar el número de registros y columnas, los tipos de datos y la presencia de valores nulos o inconsistencias.
2. Analizar la distribución de las variables: Explorar la variabilidad en precios, metros cuadrados, ubicaciones y otras características relevantes.
3. Detectar relaciones y correlaciones: Investigar posibles asociaciones entre las variables y su impacto en el precio de los inmuebles.
4. Identificar valores atípicos: Detectar posibles errores o anomalías en los datos que puedan afectar el análisis.
5. Generar insights clave: Extraer información relevante para futuros estudios o modelos de predicción del valor de los inmuebles.

Este análisis servirá como base para entender mejor la dinámica del mercado inmobiliario y podrá ser de utilidad para compradores, vendedores, inversionistas y analistas del sector.

# **_1. Librerías y configuración_**

In [1]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones
import re
#from unidecode import unidecode
import unicodedata
import datetime
import os
import math

# visuzalizaciones
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
#import matplotlib as plt
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from plotly.offline import iplot
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import missingno as msno
import gc


In [2]:
# Forzar la recolección de basura
gc.collect()

60

In [3]:
pd.set_option('display.max_info_columns',110 )
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows',80)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas
# pd.set_option('display.max_rows', 500)

# Inicializar la lista to replace
to_replace=['Null','Nan','nan','NULL',None]

# **_2. Funciones_**

In [4]:
#Funcion que agrupa las columnas por tipo de dato

def columnas_por_tipo(df):
    # Filtrar columnas por tipo de datos
    int_columns = [col for col in df.columns if df[col].dtype.name.startswith('int')]
    float_columns = [col for col in df.columns if df[col].dtype.name.startswith('float')]
    boolean_columns=[col for col in df.columns if set(df[col].unique()) <= {0, 1}]
    numeric_cols=int_columns + float_columns
    object_columns = df.select_dtypes(include='object').columns.tolist()
    category_columns = df.select_dtypes(include='category').columns.tolist()

    cat_columns=list(set(category_columns) - set(boolean_columns) )
    # Obtener todas las columnas
    all_columns = set(df.columns)
    # # Columnas de otros tipos
    other_columns = list(all_columns - set(int_columns) - set(float_columns) - set(object_columns) - set(category_columns))

    # Cuento el número de columnas de cada tipo
    num_int_columns = len(int_columns)
    num_float_columns = len(float_columns)
    num_object_columns = len(object_columns)
    num_category_columns = len(category_columns)
    num_other_columns = len(other_columns)

    # Mostrar los resultados
    print("Número de columnas de tipo entero:", num_int_columns)
    print("Columnas de tipo entero:", int_columns)
    print("Número de columnas de tipo float:", num_float_columns)
    print("Columnas de tipo float:", float_columns)
    print("Número de columnas de tipo object:", num_object_columns)
    print("Columnas de tipo object:", object_columns)
    print("Número de columnas de tipo category:", num_category_columns)
    print("Columnas de tipo category:", category_columns)
    print("Número de columnas de otros tipos:", num_other_columns)
    print("Columnas de otros tipos:", other_columns)
    return boolean_columns,numeric_cols,cat_columns,object_columns,other_columns

In [5]:

def reduce_mem_usage(df, turn_cat=False, silence=True):
    """Itera sobre todo el dataset convirtiendo cada columna en el tipo más adecuado para ahorrar memoria.

    Parameters
    ----------
    df : pd.DataFrame
        Dataframe que se quiere reducir.
    turn_cat : bool, optional
        Transformación de las columnas objeto o string a category, by default False.

    Returns
    -------
    pd.DataFrame
        Dataframe optimizado.
    """

    start_mem = df.memory_usage().sum() / 1024**2  # Memoria inicial

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object and not isinstance(col_type, pd.CategoricalDtype):
            c_min = df[col].min()
            c_max = df[col].max()

            if pd.api.types.is_numeric_dtype(df[col]):
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                else:
                    if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)

                # Conversión de float a int si no hay decimales y no hay NaN
                df[col] = df[col].fillna(0)  # Rellenar NaN antes de verificar si son enteros
                if all(df[col].apply(float.is_integer)):
                    df[col] = df[col].astype(np.int64)

        # Conversión de objetos a categoría si turn_cat es True
        if turn_cat and col_type == object:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2  # Memoria final

    if not silence:
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df


In [6]:
def graficobarrascategoricas(df, colname):
    """Genera un gráfico de barras para una variable categórica, mostrando las 10 categorías más frecuentes y agrupando el resto como 'Otros'.

    Parameters:
    df (DataFrame): El DataFrame que contiene los datos.
    colname (str): Nombre de la variable categórica.

    Returns:
    None
    """
    # Obtener las frecuencias de las categorías
    value_counts = df[colname].value_counts()

    # Seleccionar las 10 categorías más frecuentes
    top_10 = value_counts.head(10)

    # Agrupar el resto en 'Otros'
    other_count = value_counts.tail(len(value_counts) - 10).sum()  # Sumar las frecuencias de las categorías restantes
    if other_count > 0:  # Solo agrupar si hay categorías para agrupar
        otros_series = pd.Series({'Otros': other_count})
        top_10 = pd.concat([top_10, otros_series])  # Usamos pd.concat() en lugar de append

    # Crear el gráfico de barras
    return top_10

In [7]:
def plot_histogram_by_target(df, input_name, target_name, normalize = False):
    """Plots the histogram and boxplot of a numerical variable by target

    Parameters:
    input_name (str): Numeric variable's name
    target_name (str): Target variable's name

    Returns:
    None
    """
    fig = plt.figure(figsize=(15, 6))  # Ajustar el tamaño de la figura según sea necesario
    fig.suptitle(input_name, fontsize=16)


    ax1 = fig.add_subplot(121)
    sns.histplot(data=df, x=input_name, hue=target_name, alpha=.5, bins=25, ax=ax1, stat = 'density', common_norm = not normalize)

    ax2 = fig.add_subplot(122)
    sns.boxplot(data=df, x=target_name, y=input_name, ax=ax2)

    plt.tight_layout()
      # Asegurar que el título de la figura no se superponga
    plt.subplots_adjust(top=0.85)

In [8]:

def plot_barplot_by_target(df, input_name, target_name, stacked=False):
    """
    Grafica un gráfico de barras para una variable categórica en función de una variable objetivo.

    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame con los datos.
    input_name : str
        Nombre de la columna categórica a analizar.
    target_name : str
        Nombre de la columna objetivo (target).
    stacked : bool, opcional, default=False
        Si se debe apilar el gráfico de barras.
    """
    # Crear una tabla de contingencia
    crosstab = pd.crosstab(df[input_name], df[target_name])

    # Crear un gráfico de barras apiladas usando Plotly
    fig = px.bar(crosstab,
                 barmode='stack',  # Esto apila las barras
                # color=crosstab.columns,  # Colores por las columnas
                 title=f'Distribución de {input_name} por {target_name}')

    # Mostrar el gráfico
    fig.update_layout(xaxis_title=input_name,
                      yaxis_title='Frecuencia',
                      xaxis_tickangle=-45)

    fig.show()

In [9]:


def generar_barplot_media_mediana(df, cat_vars, target_var):
    """
    Genera gráficos en dos columnas: barplots con la media y mediana
    de una variable numérica (target_var) frente a varias categóricas.

    :param df: DataFrame de pandas con los datos.
    :param cat_vars: Lista de variables categóricas.
    :param target_var: Variable numérica a analizar (por ejemplo, 'precio').
    """
    filas = len(cat_vars)  # Una fila por variable categórica
    plt.figure(figsize=(15, filas * 5))  # Ajustar tamaño según número de filas

    for i, var in enumerate(cat_vars):
        # Gráfico de la media
        plt.subplot(filas, 2, 2 * i + 1)  # Columna 1: Media
        sns.barplot(x=df[var], y=df[target_var], estimator=np.mean)
        plt.xticks(rotation=90)  # Rotar etiquetas si son largas
        plt.title(f'Media de {target_var} por {var}')
        plt.xlabel(var)
        plt.ylabel(f'Media de {target_var}')

        # Gráfico de la mediana
        plt.subplot(filas, 2, 2 * i + 2)  # Columna 2: Mediana
        sns.barplot(x=df[var], y=df[target_var], estimator=np.median)
        plt.xticks(rotation=90)
        plt.title(f'Mediana de {target_var} por {var}')
        plt.xlabel(var)
        plt.ylabel(f'Mediana de {target_var}')

    plt.tight_layout()  # Ajustar el espacio entre gráficos
    plt.show()


In [10]:
def categorize_ano_construccion(value):
    # Si el valor es NaN, lo dejamos como NaN
    if pd.isna(value):
        return np.nan
    try:
        # Convertimos el valor a float si es numérico
        value = float(value)

        # Asignamos el valor a las franjas de años correspondientes
        if value < 1973:
            return 'Más de 50 años'
        elif 1973 <= value <= 1993:
            return 'Entre 30 y 50 años'
        elif 1993 < value <= 2013:
            return 'Entre 10 y 30 años'
        elif 2013 < value <= 2018:
            return 'Entre 5 y 10 años'
        elif value > 2018:
            return 'Menos de 5 años'
        else:
            return np.nan  # En caso de que no cumpla con ninguna condición
    except:
        # Si el valor no es un número, lo dejamos como está
        value=str(value).strip()

        return value

In [11]:
def agrupar_antiguedad(valor):
    if valor in ['Menos de 5 años', 'Menos de 5 Anos']:
        return 'Menos de 5 años'
    elif valor in ['Menos de 10 años','Entre 5 y 10 años']:
        return 'Entre 5 y 10 años'
    elif valor in ['Entre 10 y 30 años', 'Entre 10 y 20 años', 'Entre 20 y 30 años', 'Entre 15 y 20 Anos','Entre 10 y 15 Anos', 'Entre 15 y 20 años', 'Entre 20 y 25 Anos','Entre 10 y  15 Anos']:
        return 'Entre 10 y 30 años'
    elif valor in ['Entre 30 y 50 años', 'Entre 35 y 50 Anos']:
        return 'Entre 30 y 50 años'
    elif valor in ['Más de 50 años', 'Mas de 50 anos', 'Mas de 25 Anos']:
        return 'Más de 50 años'
    else:
        return valor


# Función para calcular la mediana de m2_contr

def calcular_mediana_colconnulos(row, df, col_connulos, colcomparar1, colcomparar2):
    filtro = (df[colcomparar1] == row[colcomparar1]) & (df[colcomparar2] == row[colcomparar2])
    mediana_colconnulos = df.loc[filtro, col_connulos].median()
    return int(mediana_colconnulos) if not pd.isna(mediana_colconnulos) else mediana_colconnulos

In [12]:

def calcular_mediana_colconnulos(df, col_connulos, colcomparar1, colcomparar2):
    """
    Rellena los valores nulos de col_connulos con la media calculada
    para las filas que coincidan en colcomparar1 y colcomparar2.

    :param df: DataFrame con los datos.
    :param col_connulos: Columna con valores nulos a imputar.
    :param colcomparar1: Primera columna de comparación para el filtro.
    :param colcomparar2: Segunda columna de comparación para el filtro.
    :return: DataFrame con los valores imputados.
    """
    # Iterar sobre las filas donde col_connulos es nulo
    for idx, row in df[df[col_connulos].isna()].iterrows():
        # Filtrar el DataFrame usando colcomparar1 y colcomparar2
        filtro = (df[colcomparar1] == row[colcomparar1]) & (df[colcomparar2] == row[colcomparar2])

        # Calcular la media de col_connulos en ese subconjunto
        mediana = df.loc[filtro, col_connulos].median()

        # Imputar la media si no es NaN
        if not np.isnan(mediana):
            df.at[idx, col_connulos] = mediana

    return df


In [13]:
def extract_num(col):
    try:
        # Si el valor es un número flotante o entero, convertirlo a float (por ejemplo, 1.0 o 2)
        if isinstance(col, (int, float)):
            return float(col)

        # Si el valor es una cadena y contiene un número, extraemos el primer número
        return float(col.split()[0])
    except:
        return np.nan

In [14]:

# Función para identificar si es público o privado
def extract_texto(col, texto1, texto2):
    if isinstance(col, str):  # Solo aplicar 'lower()' si el valor es una cadena
        if texto1 in col.lower():
            return texto1
        elif texto2 in col.lower():
            return texto2
    return np.nan

In [15]:
def asignar_columna(row, diccionario,col_asignar,col_agrupar,colfiltrado,valfiltrado):
    if pd.isnull(row[col_asignar]) and row[colfiltrado] == valfiltrado and pd.notnull(row[col_agrupar]):
        return diccionario.get(row[col_agrupar], 0) + 1
    return row[col_asignar]

In [16]:
#def diccionario_mediana(df,colnulos,colagrup1,colagrup2):
 #   mediana = (df[df[colagrup1].notnull() & dfEDA[colnulos].notnull() ]
  #  .groupby([colagrup1, colagrup2])[colnulos]
   # .median().astype(int))
    #return mediana

In [17]:
def diccionario_mediana(df, colnulos, colagrup1, colagrup2):
    """
    Rellena los valores nulos de col_connulos con la media calculada
    para las filas que coincidan en colcomparar1 y colcomparar2.

    :param df: DataFrame con los datos.
    :param colnulos: Columna con valores nulos a imputar.
    :param colagrup1: Primera columna de comparación para el filtro.
    :param colagrup2: Segunda columna de comparación para el filtro.
    :return: DataFrame con los valores imputados.
    """
    mediana = (df[df[colagrup1].notnull() & df[colnulos].notnull()]  # Use df instead of dfEDA
                .groupby([colagrup1, colagrup2])[colnulos]
                .median()
                .astype('Int64'))  # Use Int64 to handle NaNs
    return mediana

In [19]:
dfEDA= pd.read_csv('./data/pisosEDA.csv',index_col=False)
#dfEDA= pd.read_csv('./pisosEDA.csv',index_col=False)

FileNotFoundError: [Errno 2] No such file or directory: './data/pisosEDA.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **_3. Carga de datos_**

In [ ]:
print(f"El DataFrame dfEDA tiene {dfEDA.shape[0]} filas y {dfEDA.shape[1]} columnas.")

In [ ]:
# Compramos que la columna url tiene valores unicos
dfEDA['url'].nunique() == len(dfEDA)

In [ ]:
dfEDA.columns

# **_4. Exploración de datos_**

In [ ]:
dfEDA.head(2)

In [ ]:
dfEDA.describe(include='all').T

In [ ]:
# Tratar las columnas para quitar espacios si tuvieran y poner el nombre de las columnas en minúsculas
# para tener desde el primer momento siempre el mismo nombre de columnas.
dfEDA.columns = [col.strip().lower() for col in dfEDA.columns]

In [ ]:
dfEDA.columns.tolist()

## 4.1 Tipo de datos

In [ ]:
# Función `columnas_por_tipo` que muestra un resumen de los tipo de columnas que hay en el dataframe
# boolean_columns, numeric_cols, cat_columns, object_columns, other_columns
cols_bool, cols_num, cols_cat, cols_obj, cols_other = columnas_por_tipo(dfEDA)
print("Booleans:", cols_bool)
print("Numéricas:", cols_num)
print("Categóricas:", cols_cat)
print("Objeto:", cols_obj)
print("Otras:", cols_other)

## 4.2 Identificacion de valores nulos


Identificación de Valores Nulos

Los valores nulos pueden afectar los análisis estadísticos y las visualizaciones. Para detectarlos, revisaremos:
- La cantidad de valores nulos por columna.
- El porcentaje de valores nulos en relación con el total de datos.
- Visualización de los valores nulos para identificar patrones.

### 4.2.1 Número de valores nulos por columna

In [ ]:
# Verificar valores nulos por columna
valores_nulos = dfEDA.isnull().sum()

print("Valores nulos por columna:")
print(valores_nulos[valores_nulos > 0])

### 4.2.2 Porcentaje de nulos por columna

In [ ]:
# Ver porcentaje de valores nulos
porcentaje_nulos = (dfEDA.isnull().sum() / len(dfEDA)) * 100

print("\nPorcentaje de valores nulos por columna:")
print(porcentaje_nulos[porcentaje_nulos > 0])

## 4.2.3 Visulización de nulos

In [ ]:

# Convertir valores nulos en 1 y valores no nulos en 0
df_nulos = dfEDA.isnull().astype(int)

# Graficar con imshow (MUCHO MÁS RÁPIDO)
fig = px.imshow(df_nulos, labels={'x': 'Columnas', 'y': 'Índices', 'color': 'Nulo'},
                color_continuous_scale='blues', title="Mapa de Valores Nulos")
#silvia comento los graficos para que no pese, quitar el comentario para analizarlos
#fig.show()


#### 🚩
Explicación de las gráficas
En las partes resaltadras con mayor intensidad de color hacen referencia a las variables/ columnas con mayor cantidad de valores nulos. Gracias a ello, podemos ver de manera clara si los nulos se encuentran al principio, al final , entre medio o algun otro lugar de la columna. Tambien podemos notar que especialmente las variables que empiezan con la palabra carpiteria y cancha_tenis tienen una gran cantidad de nulos al igual que otras variables.

## 4.3 Identificación de datos duplicados

Identificación de Datos Duplicados
- Los registros duplicados pueden distorsionar el análisis y deben ser detectados para evaluar si deben eliminarse.

In [ ]:
duplicados = dfEDA.duplicated().sum()

print(f"\nNúmero de registros duplicados: {duplicados}")

In [ ]:
# Mostrar ejemplos de registros duplicados (si existen)
if duplicados > 0:
    print("\nEjemplo de registros duplicados:")
    display(dfEDA[dfEDA.duplicated()].head())

# **_5. Variable target_**


In [ ]:
# Distribución de la variable target en porcentaje
dfEDA["precio"].value_counts(normalize = True)

In [ ]:
dfEDA.precio.describe(percentiles=[0.15,0.3,0.45,0.55,0.75,0.9])

## 5.1 Valores atípicos

In [ ]:
# Detectar valores atípicos con IQR
Q1 = dfEDA['precio'].quantile(0.25)
Q3 = dfEDA['precio'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Filtrar valores atípicos
outliers = dfEDA[(dfEDA['precio'] < limite_inferior) | (dfEDA['precio'] > limite_superior)]
print(f'Cantidad de valores atípicos en precio: {len(outliers)}')

In [ ]:
dfEDA.query("precio<50000 or precio>9000000")

**_Se han revisado las web de anuncios con precios atipicos y son correctos_**

In [ ]:
dfEDA.drop_duplicates(keep='first', inplace=True)

# 5.2 Distribución

In [ ]:
# HISTOGRAMA
fig = px.histogram(dfEDA,x="precio", nbins=100,title='Precio en Euros', labels={'value': 'Precio'})
fig.show()

#### 🚩
Explicación de las gráficas

En el grafico mostrado se puede visualizar claramente la distribucion de las viviendas segun su precio. Gracias a ello podemos percatarnos que hay una notable mayor cantidad de viviendas con un rango de 200 k - 400 k euros a a diferencia de las demas viviendas donde la cantidad de las mismas empieza a disminuir. Tambien se puede notar que hay varios outliers o en otras palabras viviendas que salen considerablemente de donde se encuentra el mayor numero de viviendas teniendo un precio sumamente mayor al de donde se encuentran el mayor numero de viviendas.  En conclusion, el mayor numero de viviendas se concentra en valores menores a 1 millon de euros y tenemos viviendas con precios que pueden llegar a ser mayores a 18 millones de euros lo que es algo poco comun.

# **_6. Análisis estadístico inicial_**

### **_6.1 Análisis univariado_**
El análisis univariado permite estudiar la distribución de cada variable por separado, identificando patrones, valores atípicos y posibles problemas en los datos.

#### **_6.1.1 Variables numéricas_**
Las variables numéricas se pueden analizar mediante histogramas para observar su distribución y boxplots o gráficos de violín para detectar posibles valores atípicos.

In [ ]:
#Gráfico con histogramas y violín

# Parámetros
var_porfila = 3  # Número de variables por fila
graf_porvariable = 2  # Histograma y violín
tot_var = len(cols_num)

# Calcular el número de filas y columnas
n_cols = var_porfila * graf_porvariable  # Total de columnas por fila
n_rows = math.ceil(tot_var / var_porfila)  # Número de filas necesarias

# Crear subplots dinámicamente
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(20, 3 * n_rows))
fig.suptitle('Histogramas y Gráficos de Violín', fontsize=16)

if n_rows == 1:
    axes = axes.reshape(1, -1)  # Convierte a 2D si hay solo una fila

# Añadir gráficos
for i, col in enumerate(cols_num):
    row = i // var_porfila  # Fila correspondiente
    col_hist = (i % var_porfila) * graf_porvariable  # Columna del histograma
    col_violin = col_hist + 1  # Columna del gráfico de violín

    # Histograma
    sns.histplot(dfEDA[col].dropna(), bins=30, ax=axes[row, col_hist])
    axes[row, col_hist].set_title(f'{col}', fontsize=10)
    axes[row, col_hist].set_xlabel("")
    axes[row, col_hist].set_ylabel("Frecuencia")

    # Gráfico de violín

    sns.violinplot(y=dfEDA[col].dropna(), ax=axes[row, col_violin], inner="stick", color='green', alpha=0.7)

    axes[row, col_violin].set_title(f'{col}', fontsize=10)
    axes[row, col_violin].set_ylabel("")
    axes[row, col_violin].set_xlabel("")

# Ajustar el diseño
plt.tight_layout(rect=[0, 0, 1, 0.96])
#silvia comento los graficos para que no pese, quitar el comentario para analizarlos
#fig.show()


#### 🚩
Interpretación de las gráficas.

##############################################
##################################3333
######################
Silvia: Aqui jhay que revasar las graficas y decir que variables llaman la atención por algun motivo, para tenerlo en cuenta mas adelante.El comentario de abajo no aporta nada al analisis del proyecto.

En la visualizacion mostrada se puede apreciar claramente las distribuciones de las variables numericas/ columnas con valores numericos. Se puede apreciar que hay varias columnas con valores binarios o sea que solo manejan 0 y 1 como valores. Tambien se puede notar que hay algunas variables principalmente las binarias, que en su mayoria estan compuestas por valores igual a 0, lo que significa que no nos aportan mucha informacion y deberiamos analizar que tan importante es esa poca informacion que nos aportan para ver si se sigue conservando. Dentro de las demas variables se pueden notar distribuciones mas variadas.

#### **_6.1.2 Variables categóricas_**
Para las variables categóricas, los gráficos de barras ayudan a visualizar la frecuencia de cada categoría.

In [ ]:
# Convertimos a categoricas las siguientes variables, barrio, distrito,letra_ce, tipologia,estado,tipo_inmueble,cocina,tipo_suelo,tipo_fachada,agua_caliente,grupo_cocina,
#letra_emisiones,tipo_calefaccion,energia_calefaccioninstalacion_calefaccion

cols = ["barrio", "distrito", "letra_ce", "tipologia", "estado", "tipo_inmueble",
        "cocina", "tipo_suelo", "tipo_fachada", "agua_caliente", "grupo_cocina",
        "letra_emisiones", "tipo_calefaccion", "energia_calefaccion", "instalacion_calefaccion"]

for c in cols:
    # Convertir 'categoría' a categórica
    dfEDA[c] = dfEDA[c].astype('category')

In [ ]:
# Función `columnas_por_tipo` que muestra un resumen de los tipo de columnas que hay en el dataframe
# boolean_columns, numeric_cols, cat_columns, object_columns, other_columns
cols_bool, cols_num, cols_cat, cols_obj, cols_other = columnas_por_tipo(dfEDA)
print("Booleans:", cols_bool)
print("Numéricas:", cols_num)
print("Categóricas:", cols_cat)
print("Objeto:", cols_obj)
print("Otras:", cols_other)

In [ ]:
# Definir el número de gráficos por fila
num_por_fila = 3
listacol=cols_cat
# Calcular el número de filas necesarias
num_filas = (len(listacol) + num_por_fila - 1) // num_por_fila  # Redondea hacia arriba para cubrir todos los gráficos

# Crear la figura con subplots
fig, axes = plt.subplots(nrows=num_filas, ncols=num_por_fila, figsize=(15, 5 * num_filas))
fig.suptitle("Frecuencia de Categorizaciones", fontsize=16)

 # Asegurarse de que `axes` es 2D
#axes = axes.reshape(num_filas, num_por_fila)
#Bucle para crear los gráficos en cada fila
for i, col in enumerate(listacol):
    # Obtener la fila y columna correspondiente
    row = i // num_por_fila
    col_idx = i % num_por_fila

    # Crear el gráfico de barras
    top_10 = graficobarrascategoricas(dfEDA, col)

    # Dibujar el gráfico de barras
    top_10.plot(kind='bar', ax=axes[row, col_idx])

    # Configurar los títulos y etiquetas
    axes[row, col_idx].set_title(f'{col}', fontsize=12)
    axes[row, col_idx].set_xlabel("")
    axes[row, col_idx].set_ylabel("Frecuencia")

    # Rotar las etiquetas del eje X
    axes[row, col_idx].tick_params(axis='x', rotation=45)

    # Ajustar el diseño
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Ajustar el diseño para que no se superpongan

#silvia comento los graficos para que no pese, quitar el comentario para analizarlos
#fig.show()





### **_6.2 Análisis bivariado_**
El análisis bivariado nos permite estudiar la relación entre dos variables, ya sean numéricas o categóricas.

#### **_6.2.2 Variables numéricas_**


#### Pairplots para ver la relación entre algunas de las variables numericas

In [ ]:
# Pairplot de variables numéricas claves
sns.pairplot(dfEDA[['precio', 'm2_constr', 'cod_distrito','renta bruta media por hogar']])




##### SLVIA , LOS GRAFICOS HAY QUE EXPLICARLOS CON ALGO SIMILAR A ESTO
Explicacion de la grafica

1. precio vs. m2_constr:Hay una correlación positiva, a medida que aumenta el tamaño construido (metros cuadrados), el precio tiende a incrementarse. La dispersión  muestra una nube de puntos con una tendencia ascendente.

2. precio vs. cod_distrito:La relación entre precio y código de distrito es menos clara. Es probable que el gráfico refleje precios variados dentro de cada distrito,y que el  distrito afecte, pero de manera categórica. Será necesario analizar la comparación de medias por distrito.

3. precio vs. renta bruta media por hogar: Hay una gran dispersión en los puntos por lo que la distribución no parece lineal.
Podemos ver acumulación de puntos en la parte inferior izquierda , lo que indicaria que las viviendas con precios bajos estan asociados a las rentas más bajas.En las zonas con rentas altas parece haber algo más de variabilidad con algunos valores extremos. Aunque hay dispersión, es posible que exista una relación positiva entre la renta media por hogar y el precioprecio de vivienda.


#### Matriz de correlación y heatmap

Utilizamos la matriz de correlación para  identificar relaciones entre variables numéricas.

In [ ]:
df_corr=dfEDA.corr(numeric_only=True).copy()
#silvia comento los graficos para que no pese, quitar el comentario para analizarlos

#df_corr.style.background_gradient(cmap="Greens",axis=None)


In [ ]:


# Seleccionar solo las columnas numéricas
numericas = dfEDA.select_dtypes(include=[np.number])


# Calcular la correlación de 'var1' con las demás columnas numéricas
correlaciones = numericas.corrwith(numericas['precio'])

print("Correlación de 'var1' con las demás variables numéricas:")
print(correlaciones)


#### 🚩
Explicación de la gráfica

Se puede apreciar una especial correlacion entre la variable precio con la de dormitorio m2_utiles, renta bruta media por hogar, renta bruta media por persona, mediana de la renta por unidad de consumo, renta neta media por hogar, renta neta media por persona. Las demas variables cuentan con una relacion hacia la variable precio mucho mas baja.


In [ ]:
# Distribución de variables numéricas


# # Selecciono un subconjunto de variables numericas para hacer el estudio
# cols= ['m2_constr', 'cod_barrio','cod_distrito']
# for c in cols:
#     plot_histogram_by_target(df = dfEDA.sort_values("precio"), input_name = c, target_name = 'precio', normalize = True)

#### 🚩
Explicación de las gráficas


#### **_6.2.2 Variables categoricas_**


Analizamos la relación de la variable target con las varaibles categoricas


In [ ]:
#silvia comento los graficos para que no pese, quitar el comentario para analizarlos

#generar_barplot_media_mediana(dfEDA, cols_cat, 'precio')

#### 🚩
Explicación de las gráficas

En esta grafica se puede aprecias claramente la distriburcion e influencia de el distrito sobre el precio. Logrando asi comprender como conclusion que el precio cambiara significativamente segun el distrito donde quieras vivir.

# **_6. Limpieza de datos_**

In [ ]:
dfEDA["distrito"].value_counts(dropna=False)

### **_6.1 Eliminación de duplicados_**

In [ ]:
# Comprobamos si hay filas duplicadas
len(dfEDA.drop_duplicates()) / len(dfEDA)

### **_6.2 Tratamiento de nulos_**
Dependiendo de la cantidad de valores nulos y duplicados detectados, se pueden tomar las siguientes acciones:
- Eliminar registros con muchos valores nulos si afectan el análisis.
- Imputar valores faltantes
- Eliminar registros duplicados si no aportan valor al análisis.

In [ ]:
for col in dfEDA.columns:
    porcentaje_nulos = dfEDA[col].isnull().mean() * 100
    if porcentaje_nulos > 90:
        print(f'Columna {col}: {porcentaje_nulos:.2f}% nulos')

In [ ]:
# Comprobamos si hay columnas con un 95% de los datos iguales, que no sean columnas con 0 y 1, en ese caso se eliminarán estas columnas
[col for col in dfEDA.columns if dfEDA[col].value_counts(normalize=True).max() >= 0.95]

In [ ]:
#Solo tenemos el caso de la columnas instalación calefaccion que tiene mas del 95% de los valores 'Radiadores'
dfEDA["instalacion_calefaccion"].value_counts(dropna=False)

## 🚩
REVISAR SI VAMOS A ELIMINAR CON ESTOS UMBRLES, DE MOMENTO LO COMENTO

In [ ]:
# Eliminamos LAS COLUMNAS con un porcentaje de nulos mayor del 65 % y las columnas desequilibradas en las que una categoría contiene el 95% o mas de los datos.
# umbral = len(dfEDA) * 0.35

# Eliminar columnas con más del 90% de valores nulos
# dfEDA = dfEDA.dropna(axis=1, thresh=umbral)
# dfEDA = dfEDA.drop(columns=[col for col in dfEDA.columns if dfEDA[col].value_counts(normalize=True).max() >= 0.95])
# dfEDA.shape

In [ ]:
# Comprobar las columnas que tenen nulos
for col in dfEDA.columns:
    porcentaje_nulos = dfEDA[col].isnull().mean() * 100
    if porcentaje_nulos > 0:
        print(f'Columna {col}: {porcentaje_nulos:.2f}% nulos')

### **_6.2.1 Imputación de nulos de variables numéricas_**

Agrupación en columna tipologia necesaria para hacer la imputacion de nulos de algunas columnas

#### **_Tipologia_**

In [ ]:
# Diccionario de mapeo para agrupar valores
mapping = {
    'Piso': 'Piso',
    'Apartamento': 'Piso',
    'Ático': 'Atico',
    'Atico': 'Atico',
    'Dúplex': 'Piso',
    'Duplex': 'Piso',
    'Estudio': 'Piso',
    'Loft': 'Piso',
    'Chalet': 'Chalet',
    'Chalet Adosado': 'Chalet',
    'Chalet Pareado': 'Chalet',
    'Adosado': 'Chalet',
    'Pareado': 'Chalet',
    'Villa': 'Chalet',
    'Bungalow': 'Chalet',
    'Casa': 'Chalet'
}

# Agrupar valores en la columna 'tipo_propiedad'
dfEDA['tipologia_agr'] = dfEDA['tipologia'].replace(mapping)
dfEDA["tipologia_agr"].value_counts(dropna=False)



#### Nulos  dormitorios


In [ ]:
dfEDA.query("dormitorios.isnull()")[["url","m2_constr","precio","dormitorios","distrito"]]

In [ ]:
# Establecer 'dormitorios' a 0 donde es nulo y la 'tipologia' es 'Estudio'
dfEDA.loc[(dfEDA['tipologia'] == 'Estudio') & dfEDA['dormitorios'].isnull(), 'dormitorios'] = 0

In [ ]:
dfEDA = calcular_mediana_colconnulos(dfEDA, 'dormitorios', 'distrito', 'm2_constr')
#comprobar por que depues de aplicar la funcion siguen quedando nulos, puede ser por que no haya vivendas para hacer la mediana en el distrito con la misma cantidad de dormitorios


In [ ]:
dfEDA["dormitorios"].value_counts(dropna=False)

In [ ]:
dfEDA.query("dormitorios.isnull()")

#### Nulos banos

In [ ]:

# a todos los Estudios les voy a aplicar banos=1
#el mininimo de banos tiene que ser 1, en los estudios tienen 0 habitaciones y 1 baño
# Establecer 'banos' a 1 solo si tiene un valor menor que 1 o es nulo para 'Estudio'
dfEDA.loc[(dfEDA['tipologia'] == 'Estudio'), 'banos'] = 1

In [ ]:
# Identificar y mostrar valores no numéricos, excluyendo '3+'
non_numeric_values = dfEDA[~dfEDA['banos'].apply(lambda x: str(x).replace('.', '').isdigit()) & dfEDA['banos'].notnull() & (dfEDA['banos'] != '3+')]['banos']
print("Valores no numéricos en la columna 'banos':")
print (non_numeric_values)

In [ ]:
# # Convertir la columna 'banos' a numérica , salvo 3+
dfEDA['banos'] = dfEDA['banos'].apply(lambda x: pd.to_numeric(x) if x != "3+" else x)


In [ ]:
# Reemplazar los valores '3+' por 5 en la columna 'banos'
dfEDA['banos'] = dfEDA['banos'].replace('3+', 5)

In [ ]:

dfEDA["banos"].value_counts(dropna=False)

In [ ]:
# Filtrar valores no nulos y que 'banos' no sea igual a '3+', luego calcular la mediana por 'cod_barrio' y 'tipologia_agr'
#mediana_banos_barrio_tlogia = (
 #   dfEDA[dfEDA['cod_barrio'].notnull() & dfEDA['banos'].notnull() & (dfEDA['banos'] != '3+')]
  #  .groupby(['cod_barrio', 'tipologia_agr'])['banos']
  #  .median()
#)
mediana_banos_barrio_tlogia =diccionario_mediana(dfEDA,'banos','cod_barrio','tipologia_agr')
# Filtrar valores no nulos y que 'banos' no sea igual a '3+', luego calcular la mediana por 'cod_distrito' y 'tipologia_agr'
#mediana_banos_distrito_tlogia = (
 #   dfEDA[dfEDA['cod_distrito'].notnull() & dfEDA['banos'].notnull() & (dfEDA['banos'] != '3+')]
  #  .groupby(['cod_distrito', 'tipologia_agr'])['banos']
   # .median()
#)
mediana_banos_distrito_tlogia =diccionario_mediana(dfEDA,'banos','cod_distrito','tipologia_agr')
# Convertir a diccionario
mediana_banos_barrio_dict = mediana_banos_barrio_tlogia.to_dict()
mediana_banos_distrito_dict = mediana_banos_distrito_tlogia.to_dict()




In [ ]:
#Aplic a todos los qe tienen valor +3, el valor 5


In [ ]:
# Aplicar la función a las filas seleccionadas
dfEDA['banos'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_banos_barrio_dict ,'banos','cod_barrio','tipologia_agr','Chalet'), axis=1)
dfEDA['banos'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_banos_distrito_dict ,'banos','cod_barrio','tipologia_agr','Chalet'), axis=1)




dfEDA['banos'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_banos_barrio_dict ,'banos','cod_barrio','tipologia_agr','Piso'), axis=1)
dfEDA['banos'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_banos_distrito_dict ,'banos','cod_barrio','tipologia_agr','Piso'), axis=1)

In [ ]:
# Obtener la clave con el valor máximo
clave_maxima = max(mediana_banos_barrio_dict, key=mediana_banos_barrio_dict.get)
valor_maximo = mediana_banos_barrio_dict[clave_maxima]
valor_maximo

In [ ]:
# Obtener la clave con el valor máximo
clave_maxima = max(mediana_banos_distrito_dict , key=mediana_banos_distrito_dict.get)
valor_maximo = mediana_banos_distrito_dict [clave_maxima]
valor_maximo

In [ ]:
dfEDA['banos'] = pd.to_numeric(dfEDA['banos'])

In [ ]:
# Establecer 'banos' a NaN donde 'banos' es 0
#dfEDA.loc[dfEDA['banos'] == 0, 'banos'] = np.nan

In [ ]:
# Aplicar la función a las filas seleccionadas
dfEDA['banos'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_banos_barrio_dict ,'banos','cod_barrio','tipologia_agr','Chalet'), axis=1)
dfEDA['banos'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_banos_distrito_dict ,'banos','cod_distrito','tipologia_agr','Chalet'), axis=1)


dfEDA['banos'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_banos_barrio_dict ,'banos','cod_barrio','tipologia_agr','Piso'), axis=1)
dfEDA['banos'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_banos_distrito_dict ,'banos','cod_distrito','tipologia_agr','Piso'), axis=1)

dfEDA['banos'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_banos_barrio_dict ,'banos','cod_barrio','tipologia_agr','Atico'), axis=1)
dfEDA['banos'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_banos_distrito_dict ,'banos','cod_barrio','tipologia_agr','Atico'), axis=1)

In [ ]:
dfEDA["banos"].value_counts(dropna=False)

Se ha comprobadoy la vivienda https://www.pisos.com/comprar/chalet-fuencarral_el_pardo_el_pardo28023-46668989227_102100/ tiene 13 baños


In [ ]:
dfEDA['banos'] = pd.to_numeric(dfEDA['banos'])

In [ ]:
dfEDA.banos.describe()

#####  Nulos en m2_constr

In [ ]:
dfEDA.query("m2_constr.isnull() ")[['url','m2_constr',"dormitorios","precio","cod_barrio","cod_distrito"]]

In [ ]:
dfEDA["terraza"].value_counts(dropna=False)

In [ ]:
###QUEDAN VALORES NULOS EN M2_CONSTR, le damos la mediana de las viviendas del mismo distrito y mismo numero de dormitorios


# Aplicar la función para llenar solo los valores nulos en m2_constr
dfEDA['m2_constr'] = dfEDA.apply(
    #lambda row: calcular_mediana_m2_contr(row, dfEDA) if pd.isnull(row['m2_constr']) else row['m2_constr'], axis=1)
    lambda row: calcular_mediana_colconnulos(row, dfEDA,'m2_constr','distrito','dormitorios') if pd.isnull(row['m2_constr']) else row['m2_constr'], axis=1)


In [ ]:
dfEDA.query("m2_constr.isnull()")

In [ ]:
dfEDA.reset_index(drop=True, inplace=True)

In [ ]:
#m2_utiles=0.8 * m2_constr
#dfEDA["m2_utiles"] = dfEDA["m2_utiles"].fillna(0.8 * dfEDA["m2_constr"])


#### revisamos nulos de dormitorios despues de haber completado m2_constr

In [ ]:
#vuelvo a revisar dormitorios ahora que tenemos m2_constr con datos
mediana_dormitorios_barrio_tlogia =diccionario_mediana(dfEDA,'dormitorios','cod_barrio','tipologia_agr','')
mediana_dormitorios_barrio_dict = mediana_dormitorios_barrio_tlogia.to_dict()
dfEDA['dormitorios'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_dormitorios_barrio_dict ,'dormitorios','cod_barrio','tipologia_agr','Chalet'), axis=1)
dfEDA['dormitorios'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_dormitorios_barrio_dict ,'dormitorios','cod_barrio','tipologia_agr','Piso'), axis=1)
dfEDA['dormitorios'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_dormitorios_barrio_dict ,'dormitorios','cod_barrio','tipologia_agr','Atico'), axis=1)

In [ ]:
dfEDA["banos"].value_counts(dropna=False)

#### trastero

In [ ]:
dfEDA["trastero"].value_counts(dropna=False)

In [ ]:
#Considero que hay columnas que si no tienen valor el valor que les corresponde es 0. Pondremos 0 en num_garajes, amueblado, trastero
#seguir escribiendo las columnas que correspondan y aplicarlo al final


In [ ]:
#cols_nulos_acero=["num_garajes", "amueblado","trastero"]
#for col in cols_nulos_acero:
#    dfEDA[col] = dfEDA[col].fillna(0)

### **_6.2.2 Imputación de nulos de variables categóricas_**

In [ ]:
dfEDA["tipo_fachada"].value_counts(dropna=False)

In [ ]:
dfEDA["tipo_suelo"].value_counts(dropna=False)

# **_7. Feature engineering y Categorical Encoding_**

m2_utiles

In [ ]:
# para los valores nulos calculamos los m2_utiles como 0.8 de los contruidos
#tambien podriamos calcular el indice con los valores que tenemos
def calcular_utiles(m2_constr):
    return m2_constr * 0.80

#### tipologia

Se hace antes por que lo necesitamos para imputar los nulos de banos

In [ ]:
dfEDA["tipologia_agr"].value_counts(dropna=False)

#### carpinteria

In [ ]:

# Diccionario para asignar valores de eficiencia
eficiencia_carpinteria = {
    'carpinteria_exterior_vidrio/metal': 1,
    'carpinteria_exterior_vidrio/madera': 2,
    'carpinteria_exterior_vidrio/pvc': 3,
    'carpinteria_exterior_doble_vidrio/metal': 4,
    'carpinteria_exterior_doble_vidrio/madera': 5,
    'carpinteria_exterior_doble_vidrio/pvc': 6,
    'carpinteria_exterior_triple_vidrio/metal': 7,
    'carpinteria_exterior_triple_vidrio/madera': 8
}

# Crear una columna nueva con los valores de eficiencia
def calcular_eficiencia(row):
    valores = [eficiencia_carpinteria[col] for col in eficiencia_carpinteria if row[col] == 1]
    return max(valores) if valores else None

dfEDA['eficiencia_carpinteria'] = dfEDA.apply(calcular_eficiencia, axis=1)




In [ ]:
dfEDA["eficiencia_carpinteria"].value_counts(dropna=False)

In [ ]:
dfEDA["estado"].value_counts(dropna=False)

In [ ]:
#eliminamos las columnas
cols=('carpinteria_exterior_vidrio/metal','carpinteria_exterior_vidrio/madera','carpinteria_exterior_vidrio/pvc',
    'carpinteria_exterior_doble_vidrio/metal','carpinteria_exterior_doble_vidrio/madera','carpinteria_exterior_doble_vidrio/pvc',
    'carpinteria_exterior_triple_vidrio/metal','carpinteria_exterior_triple_vidrio/madera')
dfEDA=dfEDA.drop(columns=cols)

In [ ]:
# Crear un diccionario de mapeo para agrupar las categorías
estado_map = {
    "En buen estado": "Buen estado",
    "Bueno / Habitable": "Buen estado",
    "Estado Perfecto Estado": "Buen estado",
    "A estrenar": "Reformado",
    "Reformado": "Reformado",
    "A reformar": "Para reformar",
    "Estado A reformar": "Para reformar",
    "Para reformar": "Para reformar",
    "Óptimo / Reformado": "Reformado",
    "Nuevo / En construcción": "Obra nueva",
    "Estado Depende del precio": "Depende del precio",
    "Estado Obra nueva": "Obra nueva",
    "Obra nueva": "Obra nueva"
}

# Aplicar el mapeo al dataframe
dfEDA['estado'] = dfEDA['estado'].map(estado_map).fillna('Desconocido')


#### amueblado

In [ ]:
dfEDA["amueblado"].value_counts(dropna=False)

In [ ]:
# Crear un diccionario de mapeo para convertir los valores
amueblado_map = {
    "Sí": 1,
    "No": 0,
    "1.0": 1,
    "0.0": 0,
    "0": 0,
    "Sólo cocina amueblada": 0.1,
    "Parcialmente amueblado": 0.5
}

# Aplicar el mapeo a la columna "amueblado"
dfEDA['amueblado'] = dfEDA['amueblado'].map(amueblado_map).fillna(0)


### planta Revisar ######################

In [ ]:
# Se remplaza valor raro encontrado en la columna planta
dfEDA["planta"] = dfEDA["planta"].replace({" , 2", 2})

In [ ]:
# Limpiar los valores de la columna 'planta'
dfEDA['planta'] = dfEDA['planta'].str.replace(' ,', '').str.strip()
dfEDA.loc[dfEDA['planta'] == 'Entreplanta', 'planta'] = 0.5
# Sustituir "Planta baja" por 0 en la columna 'planta'
dfEDA['planta'] = dfEDA['planta'].replace('Planta baja', 0)
# Reemplazar "Semi-sótano" por "-0,5" en la columna 'planta'
dfEDA['planta'] = dfEDA['planta'].replace('Semi-sótano', '-0.5')
# Actualizar la columna planta para las filas que cumplen con las condiciones
dfEDA.loc[(dfEDA['planta'] == 'Principal') & (dfEDA['tipologia'] == 'Piso'), 'planta'] = 3
# Actualizar la columna planta para las filas que cumplen con las condiciones
dfEDA.loc[(dfEDA['planta'] == 'Principal') & (dfEDA['tipologia'] == 'Apartamento'), 'planta'] = 3


# Limpiar espacios vacíos y otros valores no numéricos
dfEDA['planta'] = dfEDA['planta'].replace({'Sótano': '', r'\(.*\)': ''}, regex=True)
# tenemos un valor que es planta Principal en un chalet o casa no aplica, ¿que valor numerico ele puedo dar?, en un piso le voy a daer el 3 por que hace referencia a pisos que no son ni bajos ni aticoa
# dfEDA['planta'] = dfEDA['planta'].str.replace(' ,', '').str.strip()
# dfEDA['planta'] = dfEDA['planta'].str.replace('1, ', '')
#elimino esta fila por que son 3 viviendas
dfEDA = dfEDA[dfEDA['planta'] != 'Planta baja, de la 1 a la 3, 5']
dfEDA = dfEDA[dfEDA['planta'] != 'De la principal a la 2']
#mas adelante les pongo el mismo valor que Atico por que a la hora de calcular el precio es similar
# Establecer 'planta' a nulo para todos los 'Chalet'
dfEDA.loc[dfEDA['tipologia_agr'] == 'Chalet', 'planta'] = np.nan
# Convierte la columna 'planta' a valores numéricos, forzando errores en NaN donde no sea posible
# dfEDA['planta'] = pd.to_numeric(dfEDA['planta'])
dfEDA['planta'] = pd.to_numeric(dfEDA['planta'], errors='coerce')

#hay aticos con valor planta=1 en los datos, creo que es un error en la publicacion de la vivieda, los pongo a nulos para asignarles el valor de la ultima planta en el barrio
dfEDA.loc[(dfEDA['planta'] == 1) & (dfEDA['tipologia_agr'] == 'Atico'), 'planta'] = np.nan
#El edificio de viviendas mas alto de Madrid tiene 33 plantas, los datos con planta mayores de ese valor son erroneos por ello le voy a poner valor nulo si se cumple
# dfEDA.loc[(dfEDA['planta'] > 33) & (dfEDA['tipologia_agr'] == 'Piso' | dfEDA['tipologia_agr'] == 'Atico'), 'planta'] = np.nan
# El edificio de viviendas más alto de Madrid tiene 33 plantas, los datos con planta mayores de ese valor son erroneos, por ello les pongo valor nulo si se cumple
dfEDA.loc[(dfEDA['planta'] > 33) & (dfEDA['tipologia_agr'] == 'Piso'), 'planta'] = np.nan
dfEDA.loc[(dfEDA['planta'] > 33) & (dfEDA['tipologia_agr'] == 'Atico'), 'planta'] = np.nan

In [ ]:
dfEDA.planta.describe()

In [ ]:
dfEDA

In [ ]:

# Filtrar valores no nulos de 'cod_barrio' y 'planta' y obtener la máxima 'planta' por 'cod_barrio'
max_planta_barrio = dfEDA[dfEDA['cod_barrio'].notnull() & dfEDA['planta'].notnull()].groupby('cod_barrio')['planta'].max().astype(int)
max_planta_distrito = dfEDA[dfEDA['cod_distrito'].notnull() & dfEDA['planta'].notnull()].groupby('cod_distrito')['planta'].max().astype(int)
mediana_planta_barrio = dfEDA[dfEDA['cod_barrio'].notnull() & dfEDA['planta'].notnull()].groupby('cod_barrio')['planta'].median().astype(int)
mediana_planta_distrito = dfEDA[dfEDA['cod_distrito'].notnull() & dfEDA['planta'].notnull()].groupby('cod_distrito')['planta'].median().astype(int)
# Convertir el resultado a un diccionario
max_planta_barrio_dict = max_planta_barrio.to_dict()
max_planta_distrito_dict = max_planta_distrito.to_dict()
mediana_planta_barrio_dict = mediana_planta_barrio.to_dict()
mediana_planta_distrito_dict = mediana_planta_distrito.to_dict()

In [ ]:
dfEDA["planta"].value_counts(dropna=False)

In [ ]:

# Aplicar la función a las filas seleccionadas
dfEDA['planta'] = dfEDA.apply(lambda row: asignar_columna(row, max_planta_barrio_dict,'planta','cod_barrio','tipologia_agr','Atico'), axis=1)
# si el cod_barrio es nulo, tomo el aximo del distrito
dfEDA['planta'] = dfEDA.apply(lambda row: asignar_columna(row, max_planta_distrito_dict,'planta','cod_distrito','tipologia_agr','Atico'), axis=1)
# voy a cosiderar los chalets como los aticos ya que a la hora de predecir el precio tiene valor similar en Madrid
# Actualizar el valor de planta a 10 para todas las filas donde tipologia sea Casa o Chalet
dfEDA['planta'] = dfEDA.apply(lambda row: asignar_columna(row, max_planta_barrio_dict,'planta','cod_barrio','tipologia_agr','Chalet'), axis=1)
# si el cod_barrio es nulo, tomo el aximo del distrito
dfEDA['planta'] = dfEDA.apply(lambda row: asignar_columna(row, max_planta_distrito_dict,'planta','cod_distrito','tipologia_agr','Chalet'), axis=1)

#una vez puesto la planta en los aticos , maximo valor de planta mas 1, tambien lo ponemos en los pisos, pero en este caso n es el maximo si no la mediana
dfEDA['planta'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_planta_barrio_dict,'planta','cod_barrio','tipologia_agr','Piso'), axis=1)
# si el cod_barrio es nulo, tomo el aximo del distrito
dfEDA['planta'] = dfEDA.apply(lambda row: asignar_columna(row, mediana_planta_distrito_dict,'planta','cod_distrito','tipologia_agr','Piso'), axis=1)


#### garaje

In [ ]:
# Aplicamos las funciones a la columna 'garaje'
dfEDA['num_garajes'] = dfEDA['garaje'].apply(lambda x: extract_num(x))
dfEDA['tipo_garaje'] = dfEDA['garaje'].apply(lambda x: extract_texto(x, "privado", "público"))

In [ ]:
#Eliminamos la columna garaje
dfEDA.drop(columns=['garaje'], inplace=True)

#### ano_construccion

In [ ]:
dfEDA["ano_construccion"].value_counts(dropna=False)

In [ ]:
# Aplicamos la función a la columna 'ano_construccion'
dfEDA['ano_construccion'] = dfEDA['ano_construccion'].apply(categorize_ano_construccion)

In [ ]:
# Aplicar la función al DataFrame
dfEDA['ano_construccion'] = dfEDA['ano_construccion'].apply(agrupar_antiguedad)

In [ ]:
# Verificamos los primeros registros para comprobar que la agrupación fue realizada correctamente
dfEDA['ano_construccion'].value_counts()

In [ ]:
#hago OneHotEncoding en ano_construccion

En ano_construccion se puede hacer one hot encoding, o dar un valor de 1 a 6 en función de la antiguedad

#### gastos_comunidad

In [ ]:
# Aqui hago las modificaiones necesarias en la columna ascensor para que este con los valores correctos.

# Reemplazo de valores
dfEDA["ascensor"]= dfEDA["ascensor"].replace({"Sí": 1})
dfEDA["ascensor"]= dfEDA["ascensor"].replace({"No": 0})
dfEDA["ascensor"]= dfEDA["ascensor"].replace({1.0: 1})
dfEDA["ascensor"]= dfEDA["ascensor"].replace({0.0: 0})

dfEDA["ascensor"]= dfEDA["ascensor"].replace({"1": 1})
dfEDA["ascensor"]= dfEDA["ascensor"].replace({"0": 0})

# Rellenar nulos con 0 . Lo que equivale a que no tiene ascensor.
dfEDA["ascensor"]= dfEDA["ascensor"].fillna(0)

In [ ]:
dfEDA["gastos_comunidad"].unique()

# **_Lista de nombres de columnas booleanas_**
columnas_int = [ 'terraza', 'trastero', 'ascensor', 'porteria','tiene_armario','cancha_tenis','tiene_piscina']
columnas_float=['precio','m2_constr','m2_utiles']
# **_Asignar el tipo booleano a las columnas_**
for columna in columnas_int:
    dfEDA[columna] = dfEDA[columna].astype(int)
for columna in columnas_float:
    dfEDA[columna] = dfEDA[columna].astype(float)

In [ ]:
# Eliminamos las columnas:
colseliminar=['carpinteria_exterior_vidrio/metal','carpinteria_exterior_vidrio/madera', 'carpinteria_exterior_vidrio/pvc',
    'carpinteria_exterior_doble_vidrio/metal',    'carpinteria_exterior_doble_vidrio/madera',    'carpinteria_exterior_doble_vidrio/pvc',
    'carpinteria_exterior_triple_vidrio/metal',    'carpinteria_exterior_triple_vidrio/madera']
dfEDA = dfEDA.drop(columns=colseliminar)

In [ ]:
dfEDA["dormitorios"].value_counts(dropna=False)

In [ ]:
dfEDA=reduce_mem_usage(dfEDA, turn_cat=False, silence=False)

In [ ]:
dfEDA.info()

In [ ]:
dfEDA["dormitorios"].value_counts(dropna=False)

### **_Calculo de nuevas columnas / Feature Engineering_**

In [ ]:
#alculamos Euros_M2= precio/m2_contr

dfEDA["euros_m2"]=dfEDA["precio"]/dfEDA["m2_constr"]

#### precio_medio_barrio

In [ ]:
#dfEDA=reduce_mem_usage(dfEDA, turn_cat=False, silence=True)

In [ ]:
dfEDA["m2_constr"].value_counts(dropna=False)

In [ ]:
dfEDA

# **_8. Análisis estadístico final_**

In [ ]:
# Chequeo de los valores descriptivos/estadisticos de las columnas numericas
dfEDA.describe().T

In [ ]:
# Chequeo de los valores unicos de la columna
dfEDA ["porteria"].unique()

In [ ]:
# Chequeo de tipos de datos de las columnas
dfEDA.info()

In [ ]:
# Modificacion de tipo de datos.


dfEDA = dfEDA.astype({
    "url": "object",
    "distrito": "category",
    "barrio": "category",
    "letrace": "string",
    "tipologia": "category",
    "estado": "category",
    "ascensor": "int64",
    "porteria": "category",
    "certificado_energetico":"category",
    "agua_caliente": "category",
    "tipo_suelo": "category",
    "tipo_fachada": "category",
    "grupo_cocina": "category",
    "clasificacion_aire":"category",
    "letra_emisiones":"category",
    "tipo_calefaccion": "category",
    "energia_calefaccion":"category",
    "instalacion_calefaccion":"category",
    "tipologia_agr":"category",
    "tipo_garaje": "category"


})

In [ ]:
dfEDA

In [ ]:
# Chequeo de valores columnas letrace
dfEDA["letrace"].unique()

In [ ]:
# Relleno de nulos
dfEDA["letrace"] = dfEDA["letrace"]. fillna("sin_registro")

In [ ]:
# Confirmacion de cambios
dfEDA["letrace"].unique()

In [ ]:
# Cambio de tipo de datos
#dfEDA["letrace"] = dfEDA["letrace"].astype("category")

In [ ]:
# Acceder a las columnas categóricas (ajustar según la estructura de la tupla)
columnas_categoricas = columnas_por_tipo(dfEDA)[1]

# Graficar la distribución de cada columna categórica y enteros
for col in columnas_categoricas:
    plt.figure(figsize=(6, 4))
    sns.countplot(x=dfEDA[col])
    plt.title(f"Distribución de {col}")
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
print(type(columnas_por_tipo(dfEDA)))  # Para ver el tipo de dato
print(columnas_por_tipo(dfEDA))

In [ ]:
dfEDA["barrio"].unique()

In [ ]:
#  Evitamos que se trunque con value_counts()
pd.set_option('display.max_rows', None)
# Regresamos a que se trunque
# pd.reset_option('display.max_rows', None)

In [ ]:
dfEDA["barrio"].value_counts()

In [ ]:
# tipologia está agrupada en tipologia_agrup
dfEDA["tipologia"].value_counts()

In [ ]:
# conversion del tipo de dato
dfEDA['barrio'] = dfEDA['barrio'].fillna('').astype(str)


In [ ]:
#No es necesario
# Eliminar espacios al principio y al final, y reemplazar dobles espacios en las columnas 'barrio' y 'tipologia'
#dfEDA['barrio'] = dfEDA['barrio'].str.strip().str.replace(r'\s+', '', regex=True)
#dfEDA['tipologia'] = dfEDA['tipologia'].str.strip().str.replace(r'\s+', '', regex=True)

# Eliminar acentos en las columnas 'barrio' y 'tipologia'
#dfEDA['barrio'] = dfEDA['barrio'].apply(unidecode)
#dfEDA['tipologia'] = dfEDA['tipologia'].apply(unidecode)

In [ ]:
dfEDA["barrio"].unique()

In [ ]:
dfEDA["barrio"].value_counts()

In [ ]:
#dfEDA["tipologia"].unique()

In [ ]:
dfEDA

Puntos a realizar...

*  Normalizar los valores en barrio. Hecho
*  Normalizar los valores en Tipologia.Hecho
*  Rellenar nulos en las columnas  para hacer el Clustering...
* Analizar de manera mas profunda feature engineering .

In [ ]:
# Forzar la recolección de basura
gc.collect()

In [ ]:
dfEDA.to_csv("dfEDA_final.csv",index='False')

# **_9. Dataset para generación del modelo_**